In [1]:
dict = {'Equal':1, 'Slightly Higher':3, 'Moderately Higher':5, 'Quite Higher':7, 'Extremely Higher':9, 
        'Slightly Lower':1/3, 'Moderately Lower':1/5, 'Quite Lower':1/7, 'Extremely Lower':1/9}

In [2]:
cols = ['Distance to Major Roads', 'Distance to Urban Centre', 'Distance to hospitals', 'Distance to warehouse/distribution structure', 'Slope of the area', 'Type of terrain', 'Availability of electricity', 'Availability of water', 'Distance to railway stations']

In [3]:
num = {}
for i in range(len(cols)):
    num[cols[i]] = i
num

{'Distance to Major Roads': 0,
 'Distance to Urban Centre': 1,
 'Distance to hospitals': 2,
 'Distance to warehouse/distribution structure': 3,
 'Slope of the area': 4,
 'Type of terrain': 5,
 'Availability of electricity': 6,
 'Availability of water': 7,
 'Distance to railway stations': 8}

In [4]:
import pandas as pd
import numpy as np
import re
from scipy.stats import gmean

In [5]:
df = pd.read_excel(f'AHP (faculty response).xlsx')

In [6]:
rank = pd.read_excel('Rankings.xlsx')

In [7]:
name = pd.read_excel(f'Name of expert.xlsx')

In [8]:
name.shape

(11, 1)

In [9]:
name.columns

Index(['Name'], dtype='object')

In [10]:
df.head()

,"As compared to Distance to Major Roads, the importance of [Distance to Urban Centre]","As compared to Distance to Major Roads, the importance of [Distance to hospitals]","As compared to Distance to Major Roads, the importance of [Distance to warehouse/distribution structure]","As compared to Distance to Major Roads, the importance of [Slope of the area]","As compared to Distance to Major Roads, the importance of [Type of terrain]","As compared to Distance to Major Roads, the importance of [Availability of electricity]","As compared to Distance to Major Roads, the importance of [Availability of water]","As compared to Distance to Major Roads, the importance of [Distance to railway stations]","As compared to Distance to Urban Centre, the importance of [Distance to hospitals]","As compared to Distance to Urban Centre, the importance of [Distance to railway stations]",...,"As compared to Distance to warehouse/distribution structure, the importance of [Slope of the area]","As compared to Distance to warehouse/distribution structure, the importance of [Type of terrain]","As compared to Distance to warehouse/distribution structure, the importance of [Availability of electricity]","As compared to Distance to warehouse/distribution structure, the importance of [Availability of water]","As compared to Slope of the area, the importance of [Type of terrain]","As compared to Slope of the area, the importance of [Availability of electricity]","As compared to Slope of the area, the importance of [Availability of water]","As compared to Type of terrain, the importance of [Availability of electricity]","As compared to Type of terrain, the importance of [Availability of water]","As compared to Availability of electricity, the importance of [Availability of water]"
0,Moderately Higher,Quite Lower,Extremely Lower,Moderately Lower,Slightly Lower,Slightly Higher,Moderately Higher,Moderately Higher,Slightly Lower,Slightly Higher,...,Slightly Higher,Moderately Higher,Quite Higher,Extremely Higher,Slightly Higher,Moderately Higher,Quite Higher,Moderately Higher,Quite Higher,Moderately Higher
1,Slightly Lower,Slightly Higher,Slightly Higher,Slightly Lower,Slightly Lower,Quite Higher,Extremely Higher,Slightly Lower,Moderately Higher,Equal,...,Slightly Lower,Moderately Lower,Quite Higher,Extremely Higher,Slightly Higher,Quite Higher,Extremely Higher,Quite Higher,Extremely Higher,Extremely Higher
2,Extremely Lower,Extremely Lower,Quite Lower,Equal,Equal,Extremely Higher,Extremely Higher,Quite Lower,Extremely Lower,Quite Lower,...,Equal,Slightly Lower,Extremely Higher,Quite Higher,Equal,Slightly Higher,Moderately Higher,Extremely Higher,Quite Higher,Equal
3,Slightly Lower,Extremely Higher,Moderately Higher,Moderately Lower,Moderately Lower,Quite Lower,Quite Higher,Moderately Higher,Extremely Higher,Slightly Higher,...,Quite Lower,Moderately Lower,Slightly Lower,Extremely Higher,Moderately Lower,Moderately Higher,Extremely Higher,Moderately Higher,Extremely Higher,Extremely Higher
4,Extremely Lower,Extremely Higher,Quite Higher,Slightly Higher,Slightly Higher,Moderately Higher,Extremely Higher,Quite Lower,Extremely Higher,Quite Lower,...,Moderately Lower,Quite Lower,Slightly Lower,Extremely Higher,Moderately Lower,Quite Higher,Extremely Higher,Quite Higher,Extremely Higher,Extremely Higher


In [11]:
for j in range(df.shape[1]):
    cur_col = df.columns[j]
    for i in range(df.shape[0]):
        df[cur_col][i] = dict[df[cur_col][i]]

In [12]:
df

,"As compared to Distance to Major Roads, the importance of [Distance to Urban Centre]","As compared to Distance to Major Roads, the importance of [Distance to hospitals]","As compared to Distance to Major Roads, the importance of [Distance to warehouse/distribution structure]","As compared to Distance to Major Roads, the importance of [Slope of the area]","As compared to Distance to Major Roads, the importance of [Type of terrain]","As compared to Distance to Major Roads, the importance of [Availability of electricity]","As compared to Distance to Major Roads, the importance of [Availability of water]","As compared to Distance to Major Roads, the importance of [Distance to railway stations]","As compared to Distance to Urban Centre, the importance of [Distance to hospitals]","As compared to Distance to Urban Centre, the importance of [Distance to railway stations]",...,"As compared to Distance to warehouse/distribution structure, the importance of [Slope of the area]","As compared to Distance to warehouse/distribution structure, the importance of [Type of terrain]","As compared to Distance to warehouse/distribution structure, the importance of [Availability of electricity]","As compared to Distance to warehouse/distribution structure, the importance of [Availability of water]","As compared to Slope of the area, the importance of [Type of terrain]","As compared to Slope of the area, the importance of [Availability of electricity]","As compared to Slope of the area, the importance of [Availability of water]","As compared to Type of terrain, the importance of [Availability of electricity]","As compared to Type of terrain, the importance of [Availability of water]","As compared to Availability of electricity, the importance of [Availability of water]"
0,5,0.142857,0.111111,0.2,0.333333,3,5,5,0.333333,3,...,3,5,7,9,3,5,7,5,7,5
1,0.333333,3,3,0.333333,0.333333,7,9,0.333333,5,1,...,0.333333,0.2,7,9,3,7,9,7,9,9
2,0.111111,0.111111,0.142857,1,1,9,9,0.142857,0.111111,0.142857,...,1,0.333333,9,7,1,3,5,9,7,1
3,0.333333,9,5,0.2,0.2,0.142857,7,5,9,3,...,0.142857,0.2,0.333333,9,0.2,5,9,5,9,9
4,0.111111,9,7,3,3,5,9,0.142857,9,0.142857,...,0.2,0.142857,0.333333,9,0.2,7,9,7,9,9
5,3,3,0.142857,0.111111,0.111111,1,1,0.333333,1,0.333333,...,0.333333,0.2,7,9,1,7,9,7,9,3
6,0.333333,9,7,0.111111,0.333333,9,9,9,9,7,...,0.111111,0.142857,5,7,3,9,9,9,9,3
7,0.142857,0.333333,0.142857,0.111111,0.142857,0.142857,0.142857,0.142857,1,0.333333,...,0.142857,0.142857,0.2,0.2,0.2,0.2,0.2,1,1,1
8,0.142857,0.2,9,0.333333,3,5,7,1,0.2,0.333333,...,1,7,5,3,5,9,7,9,7,5
9,0.111111,5,3,0.2,0.2,5,7,5,0.142857,0.333333,...,0.2,0.2,5,9,0.2,9,9,0.111111,0.111111,0.142857


In [13]:
df['As compared to Distance to Major Roads, the importance of [Distance to Urban Centre]'].sum()

9.95238095238095

To get Row of AHP matrix from the given columns

In [14]:
def row(df, i):
    return df.columns[i][15:df.columns[i].find(',')]

To get Column of AHP matrix from the given columns

In [15]:
def column(df, i):
    pattern = r"\[([^\]]+)\]"
    # Find all matches
    matches = re.findall(pattern, df.columns[i])
    return matches[0]

To get Rank of attributes

In [16]:
def get_ranks(df):
    norm_sum = [0 for i in range(df.shape[0])]
    '''for column in df.columns:
        df[column] = df[column]/df[column].sum()'''
    for i in range(df.shape[0]):
        for column in df.columns:
            norm_sum[i] += df[column][i]/df.shape[1]/df[column].sum()
    df['Weights'] = norm_sum
    df['Rank(AHP)'] = df['Weights'].rank(ascending=False)
    
    return df

Consistency Ratio

In [17]:
def consistency_ratio(df):
        df['Weighted_Sum'] = 0
        n = df.shape[0]
        for i in range(n):
            cur_column = df.columns[i]
            df['Weighted_Sum'] += df[cur_column]*df['Weights'][i]
        df['Lambda'] = df['Weighted_Sum']/df['Weights'] 
        lambda_max = df['Lambda'].mean()
        consist_rat = (lambda_max-n)/(1.45*(n-1))
        df.drop(columns=['Weighted_Sum', 'Lambda'], inplace=True)
        return consist_rat

To get individual AHP matrix

In [18]:
# Define the sizeqetyu76reqa
size = 9
indi = []
dataframes = []
# Create a 9x9 list with all ones using nested list comprehension
ahp = [[1 for _ in range(size)] for _ in range(size)]
for i in range(df.shape[0]):
    indi.append(ahp)

In [19]:
consistency = []
for j in range(df.shape[0]):
    for i in range(len(df.columns)):
        cur_col = df.columns[i]
        m = num[column(df,i)]
        n = num[row(df, i)]
        ahp[m][n] = df[cur_col][j]
        ahp[n][m] = 1/ahp[m][n]
    new_df = pd.DataFrame(ahp, columns=cols, index=cols)
    new_df = get_ranks(new_df)
    consistency.append(consistency_ratio(new_df))
    dataframes.append(new_df)

C:\Users\rajan\AppData\Local\Temp\ipykernel_30056\3132218427.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  norm_sum[i] += df[column][i]/df.shape[1]/df[column].sum()
C:\Users\rajan\AppData\Local\Temp\ipykernel_30056\3455098219.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['Weighted_Sum'] += df[cur_column]*df['Weights'][i]
C:\Users\rajan\AppData\Local\Temp\ipykernel_30056\3132218427.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `se

In [20]:
consistency

[0.20508259219196256,
 0.17292204320878057,
 1.0008472618761741,
 0.2413938398395867,
 0.37596457030956315,
 0.11545181576809,
 0.1794530049999412,
 0.1866325754696719,
 0.30914700059586836,
 1.3804883770693492,
 0.30776944039694565]

In [21]:
# Combine the DataFrames using pd.concat() with keys
individual_df = pd.concat(dataframes, keys=name['Name'])

In [22]:
individual_df.to_excel('Individual_AHP.xlsx')

To get combined AHP matrix

In [24]:
combined_ahp = np.eye(9)

In [25]:
for i in range(len(df.columns)):
        cur_col = df.columns[i]
        m = num[column(df,i)]
        n = num[row(df, i)]
        df[cur_col] = df[cur_col].astype(float)
        combined_ahp[m][n] = gmean(df[cur_col])
        combined_ahp[n][m] = 1/combined_ahp[m][n]

In [26]:
combined_ahp

array([[1.        , 3.02331499, 0.83848737, 0.93306029, 3.61238082,
        2.57551227, 0.57026266, 0.31575376, 1.21123318],
       [0.33076276, 1.        , 0.97741224, 1.0717421 , 2.33071389,
        2.63503172, 0.81893735, 0.39724515, 1.68701577],
       [1.19262381, 1.02310975, 1.        , 1.55105852, 3.32003875,
        2.76028494, 1.27818515, 0.81893735, 3.44846211],
       [1.0717421 , 0.93306029, 0.644721  , 1.        , 2.84602219,
        2.82617961, 0.48507391, 0.2567797 , 0.76411793],
       [0.27682574, 0.42905309, 0.3012013 , 0.3513676 , 1.        ,
        1.27021976, 0.29234529, 0.25274175, 0.27887168],
       [0.38827227, 0.37950207, 0.36228144, 0.35383455, 0.78726535,
        1.        , 0.25432666, 0.22166358, 0.26426424],
       [1.75357791, 1.22109462, 0.78235927, 2.06154149, 3.42061267,
        3.93195107, 1.        , 0.34610003, 1.62298018],
       [3.16702481, 2.51733723, 1.22109462, 3.8943888 , 3.95660785,
        4.51134099, 2.8893381 , 1.        , 2.90962412],


In [27]:
Combined_AHP_df = pd.DataFrame(combined_ahp, index=cols, columns=cols)

In [28]:
Combined_AHP_df = get_ranks(Combined_AHP_df)
print(consistency_ratio(Combined_AHP_df))

0.03935657929570088


C:\Users\rajan\AppData\Local\Temp\ipykernel_30056\3132218427.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  norm_sum[i] += df[column][i]/df.shape[1]/df[column].sum()
C:\Users\rajan\AppData\Local\Temp\ipykernel_30056\3455098219.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['Weighted_Sum'] += df[cur_column]*df['Weights'][i]


In [29]:
Combined_AHP_df

,Distance to Major Roads,Distance to Urban Centre,Distance to hospitals,Distance to warehouse/distribution structure,Slope of the area,Type of terrain,Availability of electricity,Availability of water,Distance to railway stations,Weights,Rank(AHP)
Distance to Major Roads,1.000000,3.023315,0.838487,0.933060,3.612381,2.575512,0.570263,0.315754,1.211233,0.118372,4.0
Distance to Urban Centre,0.330763,1.000000,0.977412,1.071742,2.330714,2.635032,0.818937,0.397245,1.687016,0.098559,5.0
Distance to hospitals,1.192624,1.023110,1.000000,1.551059,3.320039,2.760285,1.278185,0.818937,3.448462,0.150889,2.0
Distance to warehouse/distribution structure,1.071742,0.933060,0.644721,1.000000,2.846022,2.826180,0.485074,0.256780,0.764118,0.086618,7.0
Slope of the area,0.276826,0.429053,0.301201,0.351368,1.000000,1.270220,0.292345,0.252742,0.278872,0.039155,8.0
Type of terrain,0.388272,0.379502,0.362281,0.353835,0.787265,1.000000,0.254327,0.221664,0.264264,0.037328,9.0
Availability of electricity,1.753578,1.221095,0.782359,2.061541,3.420613,3.931951,1.000000,0.346100,1.622980,0.133013,3.0
Availability of water,3.167025,2.517337,1.221095,3.894389,3.956608,4.511341,2.889338,1.000000,2.909624,0.245264,1.0
Distance to railway stations,0.825605,0.592763,0.289984,1.308699,3.585879,3.784091,0.616150,0.343687,1.000000,0.090801,6.0


In [30]:
Combined_AHP_df.to_excel('Combined_AHP.xlsx')